In [1]:
import os 

In [2]:
pwd

'd:\\projects\\AI-Tex-Summraiser\\research'

In [3]:
os.chdir("../")

In [4]:
pwd

'd:\\projects\\AI-Tex-Summraiser'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path 


In [6]:
from ai_text_summariser.constants import * 
from ai_text_summariser.utils.common import read_yaml, create_directories
from ai_text_summariser.logging import logger


In [7]:
from pathlib import Path

class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = Path(r"D:\projects\AI-Tex-Summraiser\config\config.yaml"),
        params_filepath: Path = Path(r"D:\projects\AI-Tex-Summraiser\params.ymal")
    ):
        print("Reading config from:", config_filepath)
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        return DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )


In [8]:
import os
import urllib.request as request 
import zipfile
from ai_text_summariser.utils.common import get_size

In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

Reading config from: D:\projects\AI-Tex-Summraiser\config\config.yaml
[2025-05-21 16:14:18,853: INFO: common: yaml file: D:\projects\AI-Tex-Summraiser\config\config.yaml loaded successfully]
[2025-05-21 16:14:18,855: INFO: common: yaml file: D:\projects\AI-Tex-Summraiser\params.ymal loaded successfully]
[2025-05-21 16:14:18,856: INFO: common: created directory at: artifacts]
[2025-05-21 16:14:18,856: INFO: common: created directory at: artifacts/data_ingestion]
[2025-05-21 16:14:22,371: INFO: 1434958058: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: BD38:F5624:3E2C96:808

In [101]:
print(logger)


<Logger textsummrasierlogger (INFO)>
